# -----------------------------------------------------------------------------
PROJET/PURPOSE:
The UDACITY DATA SCIENCE project consisting in writing a Data Science Blog Post
It's the opportunity to dive in details in a hot topic.

Considering the rebound of COVID-19 cases in France these last weeks, I decided
 to see by myself the progress of the COVID-19 in France with figures directly
 reported by the hospitals via institutional site.
 I was requested to handle this project in python, a first through a dedicated
  Notebook. It is available here-below.

# -----------------------------------------------------------------------------
PROJECT'S TITLE
Monitoring per department of the COVID-19 situation in France (16-Dec-2022)
Rebound of contamination and hospitalization in the last month.

In [247]:
# -----------------------------------------------------------------------------
# import libraries
import sys
import re
import numpy as np
import pandas as pd
import pygal

from pygal_maps_fr import maps
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# requires installing
# https://gtk-win.sourceforge.io/home/index.php/Main/Downloads

# -----------------------------------------------------------------------------
DATA UNDERSTANDING - Data access:
Data are stored in the csv file named "COVID19_France_data.csv". It is 
 provided (below the GITHUB uploading 30Mo-limit) with "," used as separator.
This csv data file is read and transpose in dataframe via the python script
 provided on GITHUB.

The source web site indicates that data have been gathered along the time 
from various format and content.
The last version of these data is available on the following web site:
https://www.data.gouv.fr/fr/datasets/synthese-des-indicateurs-de-suivi-de-lepidemie-covid-19/
It is mentioned that this file contains all figures of the main indicators 
related to COVID-19 in France.

In [248]:
# -----------------------------------------------------------------------------
# Load the csv data file and convert it into a pandas dataframe for further use.
# Instruction: modify the filepath according to your workspace.
filepath = r"C:\Users\to202835\OneDrive - ATR\_exploitation\formation\2022\Udacity_DataScience\01_Datascience\project_01\workspace_1\project_01_main_v1.2.0\table-indicateurs-open-data-dep-2022-12-19-19h01.csv"
df = pd.read_csv(filepath.replace('\\','/'), dtype={"date": str, "dep": str,
    'lib_dep': str, 'lib_reg': str})

# data format of some labels are directly enforced from this stage to
#  to avoid message of Low memory warning.

# -----------------------------------------------------------------------------
This file contains a combination of main indicators for monitoring of the 
 COVID-19 epidemic in France since 18 March 2020. Data are provided per 
 department and per region of France. The file reports information related to
 COVID-19 testing campaign and patients at hospital.

In [249]:
# Have a view on the data
print('df keys:', np.array(df.columns))
df.head(4)

df keys: ['dep' 'date' 'reg' 'lib_dep' 'lib_reg' 'tx_pos' 'tx_incid' 'TO' 'R'
 'hosp' 'rea' 'rad' 'dchosp' 'reg_rea' 'incid_hosp' 'incid_rea'
 'incid_rad' 'incid_dchosp' 'reg_incid_rea' 'pos' 'pos_7j' 'cv_dose1']


,dep,date,reg,lib_dep,lib_reg,tx_pos,tx_incid,TO,R,hosp,rea,rad,dchosp,reg_rea,incid_hosp,incid_rea,incid_rad,incid_dchosp,reg_incid_rea,pos,pos_7j,cv_dose1
0,01,2020-03-18,84,Ain,Auvergne et Rhône-Alpes,NaN,NaN,0.062612,NaN,2,0,1,0,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,2020-03-19,84,Ain,Auvergne et Rhône-Alpes,NaN,NaN,0.132379,NaN,2,0,1,0,79,1.0,0.0,0.0,0.0,44.0,NaN,NaN,NaN
2,01,2020-03-20,84,Ain,Auvergne et Rhône-Alpes,NaN,NaN,0.155635,NaN,2,0,1,0,87,0.0,0.0,1.0,0.0,16.0,NaN,NaN,NaN
3,01,2020-03-21,84,Ain,Auvergne et Rhône-Alpes,NaN,NaN,0.173524,NaN,4,0,1,0,88,3.0,0.0,0.0,0.0,15.0,NaN,NaN,NaN


# -----------------------------------------------------------------------------
DATA UNDERSTANDING - Data Description:

- Description of data - context:

	'date'    = date (object) when the information is given: YYYY-MM-DD;
	
	'dep'     = number (str or int)of the french department;	
	
	'reg'     = number (int) of the french region;
	
	'lib_dep' = name (object) of the french department;
	
	'lib_reg' = name (object) of the french region.


- Description of data - Hospital situation:

	'hosp'         = number (int) of patients currently hospitalized due to
					  COVID-19;
					  
	'incid_hosp'   = number (float) of new hospitalized patients in the last
					  24h;
					  
	'rea' is the   = number (int) of patients currently in resuscitation or
					  intensive care unit;
					  
	'incid_rea'    = number (float) of new patients who were admitted to the 
				     resuscitation unit in the last 24h;
					 
	'rad'          = cumulative number (int) of patients who where
					  hospitalized for COVID-19 but back to home due to 
					  improvement of their health;
					  
	'incid_rad'    = number (float) of the patients back to home in the last
					  24h;
					  
	'reg_rea'       = undefined (int);
	
	'reg_incid_rea' = undefined (float).
	

- Description of data - decease due to COVID-19:

	'dchosp'       = number (int) of decease at hospital;
	
	'incid_dchosp' = number (float) of new patients deceased at the hospital
					  in the last 24h.
  

- Description of data - tests:  
  
	'pos'      = number (float) of people declared positive (D-3 date of 
	             test);
	'pos_7j'   = number (float) of people declared positive on a week
				  (D-3 data of test);
	'cv_dose1' = undefined (float).


- Description of data - COVID-19 epidemic monitoring indicators:  
  
	'tx_pos'   = Positivity rate (float) is the number of people tested
			      positive (RT-PCR or antigenic assay) for the first time in
			      the last 60 days over the number of people tested (positive
			      or negative) on a given period, without being tested 
			      positive in the last 60 days;
				  
	'tx_incid' = Incidence rate (float) is the number of people tested
			      positive (RT-PCR or antigenic assay) for the first time in
				  the last 60 days over the size of population; it is given
				  for 100 000 of inhabitants;
				  
	'TO' 	   = Occupancy rate (float) is the number of hostpitalized COVID-19
		          patients over the initial number of beds at hospital (before 
				  increase of this number).
				  
	'R' 	   = Virus replication rate (float) is the average number of people
				  that can be contaminated by a infected person.
			      R>1, epidemic is spreading. R<1, epidemic is declining.

Remark:
Though 'reg_rea', 'reg_incid_rea' and 'cv_dose1' are columns available in the
 data file, they are not described into the description notes. Consequently,
 we would remove them because they are useless without their definition.

# -----------------------------------------------------------------------------
 BUSINESS UNDERSTANDING:

 According to these data, I would propose to answer the following questions:

- Question 1: What is the rate of the population per department which is
   tested for COVID-19? According to the global health policy, it could 
   allow adjusting locally the communication for increasing test of people. 
   
- Question 2: What is the trend of admission/discharge at the hospitals? 
   It would give a trend of hospitals occupancy per department.
   
- Question 3: What is the level of degradation of hospitalized COVID-19 
   patients per department in the last 24h? It could help to quickly identify 
   where to reinforce resources for treating patients in specific department 
   when needed and increase probability to save lifes with limited means.
   
- Question 4: What is the evolution of the patients in intensive care and 
  dead in my department in the last days or weeks?   
   
- Question 5: How many people would be declared positive to the Covid-19 test
   in the upcoming week per department according to the present situation? 
   It would help to early manage logistics of patients and health resources
   and increase communication related to safety precaution.

# -----------------------------------------------------------------------------
Analysis of the need to answer the questions:
		  
According to Question 1, I would propose to compute the number of people tested
 for COVID-19 (positive or negative) over the size of the population, expressed
 for 100 000 inhabitants. So, to get current rate 'nb_test / pop (100 000 hab)',
 I will need last 'tx_incid' and 'tx_pos' of every department.

According to Question 2, I would propose to monitor the trend of hospitals
 occupancy per department. Thus I propose to monitor the rate of Input/Output 
 of patients at the hospital in the last 24h as adding new people hospitalized
 minus new people back to home and new  over the number of patients at the
 hospital: ('incid_hosp' - 'incid_rad' - 'incid_dchosp') / ('hosp' + 'rea')  

According to Question 3, I would propose to monitor in the last 24h the
 degradation of health of hospitalized people and people in intensive care unit.
 Thus I propose to monitor the rate "nb of people admitted in intensive care
 over nb of people hospitalized" ('incid_rea' / 'incid_hosp' = 'tx_rea') and
 "nb of decease over nb of people admitted in intensive car" ('incid_dchosp' /
 'incid_hosp' = 'tx_dchsop') per department.

According to Question 4, I would propose to plot the evolution of the
 patients in intensive care and dead in my department along the time in the
 six last months?    
 
According to Question 5, I would propose to build a prediction model would 
 give the number of people would be declared positive to the Covid-19 test in
 the upcoming week per department according to the current sitation. Model will
 use 'pos_7j' as target according to information information of the day of report.


In [250]:
# -----------------------------------------------------------------------------
# I start by building a function that would allows me checking some 
#  charactertics of the data set per column.
# It will help me identifying necessary processing of the raw data and
#  checking that I produce relative appropriate data (format and value).

def display_columns_info(df):
    '''
    DESCRIPTION
        Display some information per column related to the data set
    INPUT
        df is the dataframe
    OUTPUT
        nil (only display)    
    '''
    # Measure length of the column's title only for presentation purpose
    max_len = 0
    cols = np.array(df.columns)
    for c in cols:
        if len(c) > max_len:
            max_len = len(c)
    size = max_len + 2  # I will create a regulat length for label field.

    # Display information per column        
    field_1 = "column label" + ' '*(size - 11)
    field_size = 10
    field_2 = "      type "
    field_3 = "   nb uniq  rate uniq "
    field_4 = "    nb nan   rate nan "
    field_5 = "    nb inf   rate inf "
    # Introducing the results
    print(field_1 + field_2 + field_3 + field_4 + field_5)

    # Then run along the columns to perform and give the analysis per column.
    for col in cols:
        # get column's type
        col_type = df[col].dtypes
        # get number and rate of unique values
        values_uniq = np.array(pd.unique(df[col]))
        nb_uniq = len(values_uniq)
        rate_uniq = 100 * nb_uniq / df[col].shape[0]
        # get number and rate of nan values
        nb_nan = df[col].isna().sum()
        rate_nan = 100 * nb_nan / df[col].shape[0]
        # get number and rate of infinite values
        nb_inf = 0
        for val in df[col].values:
            if (val == np.inf) or (val == -np.inf):
                nb_inf += 1
        rate_inf = 100 * nb_inf / df[col].shape[0]

        # Check among 'date' values that they are all strings with a length
        #  of 10.
        mem_unexp_dates = []
        msg_unexp_dates = ''
        if col == 'date':
            for value in values_uniq:
                if (not isinstance(value, str)) or (len(value) != 10):
                    mem_unexp_dates.append(value)
            if len(mem_unexp_dates) > 0:
                msg_unexp_dates = '\tWrong date format: ' + str(mem_unexp_dates)

        # Display result
        dec = 2
        print('{}{} {}{} {}{}{}{} % {}{}{}{} % {}{}{}{} % {}'.format(
            col, ' '*(size - len(col)),
             ' ' * (field_size - len(str(col_type))), col_type,
            ' ' * (field_size - len(str(nb_uniq))), nb_uniq,
            ' ' * (field_size -1 - len(str(round(rate_uniq, dec)))), round(rate_uniq, dec),
            ' ' * (field_size - len(str(nb_nan))), nb_nan,
            ' ' * (field_size -1 - len(str(round(rate_nan, dec)))), round(rate_nan, dec),
            ' ' * (field_size - len(str(nb_inf))), nb_inf,
            ' ' * (field_size -1 - len(str(round(rate_inf, dec)))), round(rate_inf, dec),
            msg_unexp_dates)
            )

# Check of the function
display_columns_info(df)

column label          type    nb uniq  rate uniq     nb nan   rate nan     nb inf   rate inf 
dep                 object        101      0.1 %          0      0.0 %          0      0.0 % 
date                object       1007     0.99 %          0      0.0 %          0      0.0 % 
reg                  int64         18     0.02 %          0      0.0 %          0      0.0 % 
lib_dep             object        101      0.1 %          0      0.0 %          0      0.0 % 
lib_reg             object         18     0.02 %          0      0.0 %          0      0.0 % 
tx_pos             float64       4582     4.51 %       5959     5.86 %          0      0.0 % 
tx_incid           float64      50498    49.65 %       5959     5.86 %          0      0.0 % 
TO                 float64       3779     3.72 %          0      0.0 %          0      0.0 % 
R                  float64        188     0.18 %      86460    85.01 %          0      0.0 % 
hosp                 int64       1672     1.64 %          0 

# -----------------------------------------------------------------------------

Having a view on the columns' content and more specific information about their
type and values, 

# -----------------------------------------------------------------------------
DATA pre-PROCESSING

- There is only one csv data file; no need to merge several data sheets;

display of the dataframe and of the report here-above show that
 pre-processing is requested to format data for further use.

- There is no row or column that contains only nan values;
  No ACTION: only for information

- columns 'dep', 'date', 'lib_dep' and 'lib_reg' are defined as object while
  I would expect to get string (like 'reg'). So it could be interesting to 
  enforce the format of these columns.
  ACTION: DpP1

- Values of 'dep' are given in string format from '01' to '32' then in
   int format from 32 to 95 plus 971 to 976.
   For any visualization (on map or time series), i will need to clearly be
	able to compare dates and get values of a selected date: typically, get
    values at the last given date (16-Dec-2022); convert dates given in
	string into a format recognizable for time-series visualization.
   All values shall be converted in string format due to presence of values
    '2A' and '2B' rather than in int format.
   ACTION: DpP2

- Globaly, what is called rate of something is not normalized by definition.
   Actually, we note that rates like 'tw_pos' contains numerous values high
   above 1 for this reason.
   NO ACTION: only for information

- Irrelevant and outliers per columns:

	> Names of the french departments and regions can be removed after 
	   creating a dictionary to find the name from the number for further use.
	  ACTION: DpP2

	> As a reminder from description of content of the data file, undefined
	  parameters 'reg_rea', 'reg_incid_rea' and 'cv_dose1' can be removed 
      because useless without a description.
	  ACTION: DpP3
	
	> 'tx_pos' contains values higher than 1. It contains 5.9% of nan; 
			   I shall remove them before modeling.
			   ACTION: DpP4
	
	> 'tx_incid' contains 5.9% of nan; I shall remove them before modeling.
				 ACTION: DpP4
	
	> 'R' contains 85% nan; we gonna remove this column since too many values
       are missing.
	   ACTION: DpP3
	   
	> 'incid_hosp' contains 0.1% of nan; I shall remove them before modeling.
		ACTION: DpP4
	
	> 'incid_rea' contains 0.1% of nan; I shall remove them before modeling.
		ACTION: DpP4
	
	> 'incid_rad' contains 0.1% of nan; I shall remove them before modeling.
		ACTION: DpP4
	
	> 'incid_dchosp' contains 0.1% of nan; I shall remove them before modeling.
		ACTION: DpP4
	
	> 'reg_incid_rea' contains 0.1% of nan; I shall remove them before modeling.
		ACTION: DpP4
	
	> 'pos' contains 5.9% of nan; I shall remove them before modeling.
		ACTION: DpP4
	
	> 'pos_7j' contains 5.9% of nan; I shall remove them before modeling.
		ACTION: DpP4
	
	> 'cv_dose1' contains 99.9% of nan; we gonna remove this column since
       too many values are missing
	    ACTION: DpP3

# -----------------------------------------------------------------------------
Data processing - DpP1
Enforce the format of these columns 'dep', 'region', 'lib_dep' and 'lib_reg'
 to string format.

In [251]:
# -----------------------------------------------------------------------------
# DP1 - Enforce the format of these columns 'dep', 'region', 'lib_dep' and
#  'lib_reg' to string format.

# Convert into string
df['reg'] = df['reg'].astype(str)
df['dep'] = df['dep'].astype(str)
df['date'] = df['date'].astype(str)
df['lib_reg'] = df['lib_reg'].astype(str)
df['lib_dep'] = df['lib_dep'].astype(str)
# Check result of conversion
print('reg', df['reg'].dtypes)
print('dep', df['dep'].dtypes)
print('date', df['date'].dtypes)
print('lib_reg', df['lib_reg'].dtypes)
print('lib_dep', df['lib_dep'].dtypes)
# it does not seem to have the expected effect!

reg object
dep object
date object
lib_reg object
lib_dep object


# -----------------------------------------------------------------------------
Data processing - DpP2
All values shall be converted in string format due to presence of values
 '2A' and '2B' rather than in int format.
Names of the french departments and regions can be removed after 
	   creating a dictionary to find the name from the number for further use.

For visualisation, I need to convert string dates into a int value of number
 of days (timestamp). For that purpose I will split the string into the year,
 the month and the day and compute the number of passed days for every of these
 part since their 0-ref.

In [252]:
# -----------------------------------------------------------------------------
# DpP2 (start)

# Create a kind of calendar that provide the number of cumumlated passed days
#  at beginning of every month since beginning of the year.

def get_cumul_days():
    '''
    DESCRIPTION
        Return a list to ease count, month per month, passed day from the
         beginning of the year.
    INPUT
        nil
    OUTPUT
        cnt_day_at_start_month is a list that gives the nb of past days from
         beginning of the year for every month
    '''
    cnt_day_at_start_month = []

    day_by_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    cnt = 0

    for i in day_by_month:
        cnt += i 
        cnt_day_at_start_month.append(cnt)

    return cnt_day_at_start_month

# Chech the function
calendar = get_cumul_days()
print('calender:', calendar)

calender: [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]


In [253]:
# -----------------------------------------------------------------------------
# Then I use this calendar to build a function that will provide the number
#  of passed days since beginning of the year to beginning of the selected
#  month.

def get_cnt_day_at_start_month(m, calendar):
    '''
    DESCRIPTION
        get the number of past days from the beginning of year and at 
         beginning of the selected month.
    INPUT
        m is the selected month (str of int, int or float)
        calendar is a list that gives the nb of past days from beginning of the
                 year for every month
    OUTPUT
        cnt is the count of days (int) from beginning of the year at start of
         the selected month.
    '''
    cnt = calendar[int(m)-1]
    return cnt

# Check the function
# 31 days passed at beginning of february.
print('result:', get_cnt_day_at_start_month(2, calendar), 'vs expected:', 31)

result: 31 vs expected: 31


In [254]:
# -----------------------------------------------------------------------------
# Finally, I compute a timestamp from the dates as the number of days passed 
#  from 0 to the selected day such as for instances:
# 2022-06-24 => 2022 * nb_days_passed_to_this_year from year 0
#             + number_passed_days_from_beginning_of_the_year_to_beginning_of
#               this_month
#             + 24 (number of days passed since beginning of the month)
#             = 2022 * 365.24219 + 151 + 24 = 738695
# 0000-01-01 = 0 + 0 + 1 = 1


def get_timestamp_array(dates_str):
    '''
    DESCRIPTION
        Convert an np.array of dates in string format
        to a timestamp in number of days.
    INPUT
        dates_str is a np.array of dates YYYY-MM-DD in string format
    OUTPUT
        timestamp is np.array with data converted in number of days (int)
    '''
    timestamp = []
    j = 0
    for d in dates_str:
        try:
            y = float(d[:4])
            m = float(d[5:7])
            d = float(d[8:10])
            # convert the data in number of days
            cnt_days_for_months = float(get_cnt_day_at_start_month(m, calendar))
            ts = y * 365.24219 + cnt_days_for_months + d
        except:
            ts = 0.0
        ts_r = round(ts, 0)
        timestamp.append(int(ts_r))
        j += 1
    return np.array(timestamp)

test_dates = np.array(['2022-06-24', '0000-01-01'])
timestamp_arr = get_timestamp_array(test_dates)
print('expected: {} vs computed {}'.format([738695, 1], timestamp_arr))

expected: [738695, 1] vs computed [738695      1]


In [255]:
# -----------------------------------------------------------------------------
# I would like to present results per department or per region.  I will build a dictionary
# to be able to link id of a department to the name of a department.
# I build a function to be able to get a dictionary either of department or 
#  for region. Then, I will add a function specially for department.

def get_area_names(df, aera_id_label:str, aera_name_label: str):
    '''
    DESCRIPTION
        build a dictionary of a type or area (department or region) allowing
         getting a name of this area from its id
    INPUT
        df is the globale dataframe
        aera_id_label is the id (str) of the area: 'dep' (department) or
         'reg' (region)
        area_name_label is the name (str) of the area: 'lib_dep' (department)
         or 'lib_reg' (region)
    OUTPUT
        dict_areas is a dictionary: id is the key, name is the value
    '''

    dict_areas = dict()
    # get the list of area id and related names
    area_ids = np.array(df[aera_id_label])
    area_names = np.array(df[aera_name_label])
    area_id_uniq = []
    for i, area_id in enumerate(area_ids):
        if area_id not in area_id_uniq:
            area_id_uniq.append(area_id)
            dict_areas[area_id] = area_names[i]
        else:
            pass
    del df, aera_id_label, aera_name_label, area_ids, area_names, area_id_uniq
    return dict_areas

# Check the function
# - Get a dictionary of departments
dict_lib_dep = get_area_names(df, 'dep', 'lib_dep')
print('Department dictionary:\n', dict_lib_dep)
# - Get a dictionary of regions
dict_lib_reg = get_area_names(df, 'reg', 'lib_reg')
print('Region dictionary:\n', dict_lib_reg)

Department dictionary:
 {'01': 'Ain', '02': 'Aisne', '03': 'Allier', '04': 'Alpes-de-Haute-Provence', '05': 'Hautes-Alpes', '06': 'Alpes-Maritimes', '07': 'Ardèche', '08': 'Ardennes', '09': 'Ariège', '10': 'Aube', '11': 'Aude', '12': 'Aveyron', '13': 'Bouches-du-Rhône', '14': 'Calvados', '15': 'Cantal', '16': 'Charente', '17': 'Charente-Maritime', '18': 'Cher', '19': 'Corrèze', '21': "Côte-d'Or", '22': "Côtes-d'Armor", '23': 'Creuse', '24': 'Dordogne', '25': 'Doubs', '26': 'Drôme', '27': 'Eure', '28': 'Eure-et-Loir', '29': 'Finistère', '2A': 'Corse-du-Sud', '2B': 'Haute-Corse', '30': 'Gard', '31': 'Haute-Garonne', '32': 'Gers', '33': 'Gironde', '34': 'Hérault', '35': 'Ille-et-Vilaine', '36': 'Indre', '37': 'Indre-et-Loire', '38': 'Isère', '39': 'Jura', '40': 'Landes', '41': 'Loir-et-Cher', '42': 'Loire', '43': 'Haute-Loire', '44': 'Loire-Atlantique', '45': 'Loiret', '46': 'Lot', '47': 'Lot-et-Garonne', '48': 'Lozère', '49': 'Maine-et-Loire', '50': 'Manche', '51': 'Marne', '52': 'Haute-

In [256]:
# -----------------------------------------------------------------------------
# Then I build a function dedicated to the identification of a name of the
#  department according to its id. 

def get_dep_name(dep_id):
    '''
    DESCRIPTION
        give the name of a department in accordance with its id
    INPUT
        dep_id is the id (str, int or float) of the department from which we want the name
    OUTPUT
        dep_name is the name (str) of the department related to the given id
    '''
    if type(dep_id) != 'str':
        if type(dep_id) == float:
            dep_id = int(round(dep_id, 0))
        dep_id = str(dep_id)  # Ids must be string in accordance with the file
                              #  notably due to presence of deps 2A and 2B.
        if len(dep_id) == 1:
            dep_id = '0' + dep_id
    try:
        name = dict_lib_dep[dep_id]
    except:
        name = 'unknown'
    return name

# Check the function with various formats
t_str = get_dep_name('45')
t_int = get_dep_name(45)
t_float = get_dep_name(float(45.0))
t_unk = get_dep_name('604')
print('Loiret: {} - {} - {} - {}'.format(t_str, t_int, t_float, t_unk))

Loiret: Loiret - Loiret - Loiret - unknown


In [257]:
# -----------------------------------------------------------------------------
# Then I build a function dedicated to the identification of a name of the
#  region according to its id. 

def get_reg_name(reg_id):
    '''
    DESCRIPTION
        give the name of a region in accordance with its id
    INPUT
        reg_id is the id (str, int or float) of the region from which we want the name
    OUTPUT
        reg_name is the name (str) of the region related to the given id
    '''
    if type(reg_id) != 'str':
        if type(reg_id) == float:
            reg_id = int(round(reg_id, 0))
        reg_id = str(reg_id)  # Ids must be string in accordance with the file
    try:
        name = dict_lib_reg[reg_id]
    except:
        name = 'unknown'
    return name

# Check the function with various formats
t_str = get_reg_name('93')
t_int = get_reg_name(93)
t_float = get_reg_name(float(93.0))
t_unk = get_reg_name('604')
print("Provence-Alpes-Côte d'Azur: {} - {} - {} - {}".format(t_str, t_int, t_float, t_unk))

Provence-Alpes-Côte d'Azur: Provence-Alpes-Côte d'Azur - Provence-Alpes-Côte d'Azur - Provence-Alpes-Côte d'Azur - unknown


In [258]:
# -----------------------------------------------------------------------------
# Then by extension, I build a function to provide the name of departments from
#  a list of their ids.

def get_dep_names(dep_ids: list):
    '''
    DESCRIPTION
        give a list of department names according to a list of department ids
    INPUT
        dep_ids is the list of department id
    OUTPUT
        dep_names is the list of department names
    '''
    dep_names = list()
    for dep_id in dep_ids:
        dep_names.append(get_dep_name(dep_id))

    return dep_names

# Check the function
list_dep_ids = [45, '21']
print('expected: {} vs computed: {}'.format(['Loiret', "Côte-d'Or"], get_dep_names(list_dep_ids)))

expected: ['Loiret', "Côte-d'Or"] vs computed: ['Loiret', "Côte-d'Or"]


In [259]:
# -----------------------------------------------------------------------------
# In the same way, I build a function to provide the name of region from
#  a list of their ids.

def get_reg_names(reg_ids: list):
    '''
    DESCRIPTION
        give a list of region names according to a list of region ids
    INPUT
        reg_ids is the list of region id
    OUTPUT
        reg_names is the list of region names
    '''
    reg_names = list()
    for reg_id in reg_ids:
        reg_names.append(get_reg_name(reg_id))

    return reg_names

#  Check the function
list_reg_ids = [76, '27']
print('expected: {} vs computed: {}'.format(['Occitanie', 'Bourgogne et Franche-Comté'], get_reg_names(list_reg_ids)))

expected: ['Occitanie', 'Bourgogne et Franche-Comté'] vs computed: ['Occitanie', 'Bourgogne et Franche-Comté']


# -----------------------------------------------------------------------------
Data processing - DpP3
Remove following columns from the dataframe df:
 'reg_rea', 'reg_incid_rea', 'cv_dose1', 'R', 'cv_dose1'

In [260]:
# -----------------------------------------------------------------------------
# DpP3

# remove useless or incomplete columns:
size_before = df.shape
df.drop(columns=['reg_rea', 'reg_incid_rea', 'cv_dose1', 'R', 'lib_dep', 'lib_reg'], inplace=True)

# check result of the action:
print("df's size has changed from {} to {}:".format(size_before, df.shape))


df's size has changed from (101707, 22) to (101707, 16):


# -----------------------------------------------------------------------------
Data processing - DpP4
Remove empty/invalid rows from following columns from the dataframe used for
 modeling:
 'tx_pos', 'tx_incid', 'incid_hosp', 'incid_rea', 'incid_rad', 'incid_dchosp',
 'reg_incid_rea', 'pos', 'pos_7j'.
By principle, any row containing a nan or infinite value shall be removed
 from the dataframe used for modeling to work on a clean set of data.

In [261]:
# -----------------------------------------------------------------------------
# DpP4

# I remove all rows containing at least one nan (loose 5.9% of the series)
df.dropna(axis='index', how='any', inplace=True)

# Check result of the action
display_columns_info(df)

column label         type    nb uniq  rate uniq     nb nan   rate nan     nb inf   rate inf 
dep                object        101     0.11 %          0      0.0 %          0      0.0 % 
date               object        948     0.99 %          0      0.0 %          0      0.0 % 
reg                object         18     0.02 %          0      0.0 %          0      0.0 % 
tx_pos            float64       4581     4.78 %          0      0.0 %          0      0.0 % 
tx_incid          float64      50497    52.74 %          0      0.0 %          0      0.0 % 
TO                float64       3548     3.71 %          0      0.0 %          0      0.0 % 
hosp                int64       1548     1.62 %          0      0.0 %          0      0.0 % 
rea                 int64        402     0.42 %          0      0.0 %          0      0.0 % 
rad                 int64      16209    16.93 %          0      0.0 %          0      0.0 % 
dchosp              int64       4920     5.14 %          0      0.0 % 

# -----------------------------------------------------------------------------
PREPARE DATA FOR ANALYSIS

Data post-processing:

From this point, I will add new columns with computed values. Result of these
 computations may generate nan or infinite values. For this purpose, I propose
 to process computed valued after their computation to remove these unexpected
 values.
ACTION: DPP0


According to Question 1, I compute a rate 'nb_test / pop (100 000 hab)',
  dividing 'tx_pos' by 'tx_incid' (nb_positive/nb_test over
 nb_positive/nb_pop). Then I get a 'rate' which is the nb of people tested 
 for COVID-19 (positive or negative) over the size of the population, given
 for 100 000 inhabitants. I call the result column 'tx_test'.
ACTION: DPP1
 

According to Question 2, I compute the Rate of Patient-Input/Output at
 hospital as
 ('incid_hosp' - 'incid_rad' - 'incid_dchosp') / ('hosp' + 'rea') 
When necessary, I replace the sum of 'hosp' + 'rea' (denominator) by 1 when
 it equals to zero to be able to show the change. Actually, there is not a
 big difference in this context between zero people.
ACTION: DPP2
 
For this both operations, it helps to replace nan by 0 and inf values by 
the maximum of the numeric values to provide a reasonable finite a high
value instead of infinite. This is the purpose of ACTION DPP1.


According to Question 3, I compute both rates to monitor in the last 24h 
  the degradation of health of hospitalized people and people in intensive
  care unit as follows:
- Rate of people admitted in intensive care over people hospitalized:
   'incid_rea' / 'incid_hosp' = 'tx_rea'
- Rate of deceases at hospital over people admitted in intensive care:
   'incid_dchosp' / 'incid_rea' = 'tx_dchsop' 
ACTION: DPP3

For this both operations, it helps to replace nan by 0 and inf values by 
the maximum of the numeric values to provide a reasonable finite a high
value instead of infinite. This is the purpose of ACTION DPP1.


According to Question 4, there is no computation to perform. Nevertheless, 
there is a bit of preparation to capture the appropriate timeseries.
I propose to display data on the last 30 weeks, both hospitalizations and 
deceases in the same plot using the same axis to keep ratio between both.
ACTION: DPP4

According to Question 5, I built a prediction model to give 'pos_7j' as target
 according to other information, except the date because I would appreciate a
 situation only regarding the other figures with dependencies with the time.
 For the occasion I create a dataframe as a copy of the current dataframe.
 Thus I can apply additional processing to the dataframe dedicated to data
 modelization.
 It may be considered to test applying dummy values to model to identify the 
  impact and decide if it brings better result or not. According to this result
  dummy values will be or not maintain in the dataframe for modeling.
  If no dummy is used, columns with string shall be removed (due to the use
  of a Linear Regression model that does not support string).
ACTION: DPP5: it consists in:
- After DPP0, I have a dataframe specific for modeling.
- Add dummies to the specific dataframe (not activated here)
- Remove columns not compatible with modeling (str vs Linear Regression)
- Define the target (response) of the model
- split the dataset into explanatory and response data sets
- Get a trained model
- Check performance of the model (into the visualization section)

CAUTION! I shall make a copy of the cleaned dataframe for modeling before
 adding new columns.
ACTION: DPP6

# -----------------------------------------------------------------------------
Data Post-processing DPP6
copy the dataframe for further modeling without new computed parameters.

In [262]:
# -----------------------------------------------------------------------------
# Data Post-processing DPP6

# Copy df for modeling
df_lin = df.copy(deep=True)

# -----------------------------------------------------------------------------
Data Post-processing DPP0
It consist to build a function to clean the data set from unexpected values
 (nan, infinite) from the new computed data set.

In [263]:
# -----------------------------------------------------------------------------
# DDP0
# This function allows removing unexpected values from new computed columns.

def replace_invalid_data(df, label):
    '''
    DESCRIPtION
        Replace invalid data like nan or infinite by values
    INPUT
        df is the dataframe to work on
        label of the category (string) where invalid data will be replaced
    OUTPUT
        df is the dataframe modified
    '''
    # Replace nan values by 0
    df[label] = df[label].fillna(0)

    # Replace infinite (inf) values by the maximum of the serie
    maxi = df[label].max()
    df[label].replace([np.inf, -np.inf], maxi, inplace=True)  

    return df

# -----------------------------------------------------------------------------
Data Post-processing DPP1

Compute the new column 'tx_test' ('nb_test / pop (100 000 hab)') as the result
 dividing 'tx_pos' by 'tx_incid' (nb_positive/nb_test over nb_positive/nb_pop).

In [264]:
# DPP1 for answering Question 1

# Create a column with the result of 'tx_pos' / 'tx_incid' = tx_test
df['tx_test'] = df['tx_pos'].div(df['tx_incid'])  # div function allows supporting 
                                                  #  missing value (in case we miss
                                                  #  something) with fill_value
# Replace invalid data
df = replace_invalid_data(df, 'tx_test')

# Check the result
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714


In [265]:
# Test values of the result
display_columns_info(df)

column label         type    nb uniq  rate uniq     nb nan   rate nan     nb inf   rate inf 
dep                object        101     0.11 %          0      0.0 %          0      0.0 % 
date               object        948     0.99 %          0      0.0 %          0      0.0 % 
reg                object         18     0.02 %          0      0.0 %          0      0.0 % 
tx_pos            float64       4581     4.78 %          0      0.0 %          0      0.0 % 
tx_incid          float64      50497    52.74 %          0      0.0 %          0      0.0 % 
TO                float64       3548     3.71 %          0      0.0 %          0      0.0 % 
hosp                int64       1548     1.62 %          0      0.0 %          0      0.0 % 
rea                 int64        402     0.42 %          0      0.0 %          0      0.0 % 
rad                 int64      16209    16.93 %          0      0.0 %          0      0.0 % 
dchosp              int64       4920     5.14 %          0      0.0 % 

# -----------------------------------------------------------------------------
Data Post-processing DPP2

Compute 'tx_incid_IO' as the rate of Patient-Input/Output at hospital as
 ('incid_hosp' - 'incid_rad' - 'incid_dchosp') / ('hosp' + 'rea') 
When necessary, I replace the sum of 'hosp' + 'rea' (denominator) by 1 when
 it equals to zero to be able to show the change.

In [266]:
# DPP2 for answering Question 2

# monitor the trend of hospitals occupancy per department
# Create a column with the result of ('incid_hosp' - 'incid_rad' - 'incid_dchosp')
#  / ('hosp' + 'rea') = 'incid_IO'
incid_hosp = np.array(df['incid_hosp'])
incid_rad = np.array(df['incid_rad'])
incid_dchosp = np.array(df['incid_dchosp'])
hosp = np.array(df['hosp'])
rea = np.array(df['rea'])

# Avoid division by zero
people_hospitalized = np.add(hosp, rea)

# Replace 0 by 1 people to allow division 
ones = np.ones(np.shape(people_hospitalized), dtype=type(people_hospitalized))
people_hospitalized = np.where(np.absolute(people_hospitalized) < 1, ones, people_hospitalized)

tx_incid_IO = np.divide(np.subtract(incid_hosp, np.add(incid_rad, incid_dchosp)), people_hospitalized)
df['tx_incid_IO'] = tx_incid_IO

df['tx_incid_IO'] = df['tx_incid_IO'].fillna(0)  # Replace nan values by 0
df['tx_incid_IO'].replace([np.inf, -np.inf], 0, inplace=True)  #Replace infinite value (inf) by 0

# Check the result
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test,tx_incid_IO
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307,0.013605
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715,0.000000
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286,-0.014085
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143,-0.014286
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714,0.000000


In [267]:
# Test values of the result
display_columns_info(df)

column label         type    nb uniq  rate uniq     nb nan   rate nan     nb inf   rate inf 
dep                object        101     0.11 %          0      0.0 %          0      0.0 % 
date               object        948     0.99 %          0      0.0 %          0      0.0 % 
reg                object         18     0.02 %          0      0.0 %          0      0.0 % 
tx_pos            float64       4581     4.78 %          0      0.0 %          0      0.0 % 
tx_incid          float64      50497    52.74 %          0      0.0 %          0      0.0 % 
TO                float64       3548     3.71 %          0      0.0 %          0      0.0 % 
hosp                int64       1548     1.62 %          0      0.0 %          0      0.0 % 
rea                 int64        402     0.42 %          0      0.0 %          0      0.0 % 
rad                 int64      16209    16.93 %          0      0.0 %          0      0.0 % 
dchosp              int64       4920     5.14 %          0      0.0 % 

# -----------------------------------------------------------------------------
Data Post-processing DPP3

Compute:
- Rate of people admitted in intensive care over people hospitalized:
   'incid_rea' / 'incid_hosp' = 'tx_rea'
- Rate of deceases at hospital over people admitted in intensive care:
   'incid_dchosp' / 'incid_rea' = 'tx_dchsop'

In [268]:
# DPP3 for answering Question 3

# Question 3-1
# Compute Rate of people admitted in intensive care over people hospitalized
df['tx_rea'] = df['incid_rea'].div(df['incid_hosp'])
# Replace invalid data
df = replace_invalid_data(df, 'tx_rea')

# Question 3-2
# Compute Rate of deceases at hospital over people admitted in intensive care
df['tx_dchosp'] = df['incid_dchosp'].div(df['incid_rea'], fill_value=0.0)
# Replace invalid data
df = replace_invalid_data(df, 'tx_dchosp')

# Check the result
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test,tx_incid_IO,tx_rea,tx_dchosp
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307,0.013605,0.0,0.0
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715,0.000000,0.0,0.0
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286,-0.014085,0.0,inf
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143,-0.014286,0.0,inf
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714,0.000000,0.0,0.0


In [269]:
# Test values of the result
display_columns_info(df)

column label         type    nb uniq  rate uniq     nb nan   rate nan     nb inf   rate inf 
dep                object        101     0.11 %          0      0.0 %          0      0.0 % 
date               object        948     0.99 %          0      0.0 %          0      0.0 % 
reg                object         18     0.02 %          0      0.0 %          0      0.0 % 
tx_pos            float64       4581     4.78 %          0      0.0 %          0      0.0 % 
tx_incid          float64      50497    52.74 %          0      0.0 %          0      0.0 % 
TO                float64       3548     3.71 %          0      0.0 %          0      0.0 % 
hosp                int64       1548     1.62 %          0      0.0 %          0      0.0 % 
rea                 int64        402     0.42 %          0      0.0 %          0      0.0 % 
rad                 int64      16209    16.93 %          0      0.0 %          0      0.0 % 
dchosp              int64       4920     5.14 %          0      0.0 % 

# -----------------------------------------------------------------------------
Data Post-processing DPP4
Processing required for the display of values in time series.

In [270]:
# -----------------------------------------------------------------------------
# DPP4

# Add a timestamp column in df for being able to display data in time series 
#  and find most recent values per department.
df['timestamp'] = get_timestamp_array(np.array(df['date']))

# Check result
df.head()

,dep,date,reg,tx_pos,tx_incid,TO,hosp,rea,rad,dchosp,incid_hosp,incid_rea,incid_rad,incid_dchosp,pos,pos_7j,tx_test,tx_incid_IO,tx_rea,tx_dchosp,timestamp
56,01,2020-05-13,84,2.65,1.37,0.400716,139,8,315,88,6.0,0.0,4.0,0.0,9.0,9.0,1.934307,0.013605,0.0,0.0,737922
57,01,2020-05-14,84,2.32,2.74,0.377460,137,8,318,88,4.0,0.0,4.0,0.0,9.0,18.0,0.846715,0.000000,0.0,0.0,737923
58,01,2020-05-15,84,1.87,3.50,0.366726,135,7,323,89,4.0,0.0,5.0,1.0,5.0,23.0,0.534286,-0.014085,0.0,inf,737924
59,01,2020-05-16,84,1.67,3.50,0.338104,134,6,325,90,1.0,0.0,2.0,1.0,0.0,23.0,0.477143,-0.014286,0.0,inf,737925
60,01,2020-05-17,84,1.63,3.50,0.338104,133,6,326,90,1.0,0.0,1.0,0.0,0.0,23.0,0.465714,0.000000,0.0,0.0,737926


In [271]:
# -----------------------------------------------------------------------------
# DPP4

# group the data set by dep and by timestamp to ease searching results by
#  department and by time.

df.groupby(['dep', 'timestamp'])  # If no dummy applied

In [272]:
# -----------------------------------------------------------------------------
# DPP4

# Get the unique list of departments to get results for every department
deps = pd.unique(df['dep'])

# Check result
print('deps:\n', deps)

deps:
 ['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '2A' '2B' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41'
 '42' '43' '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55'
 '56' '57' '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69'
 '70' '71' '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83'
 '84' '85' '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '971' '972'
 '973' '974' '976']


In [273]:
# -----------------------------------------------------------------------------
# DPP4

# I want to display computed values at the most recent date
#  so for that purpose, I need to identify the last date and corresponding 
#  values for the selected category (column).

def get_values_at_last_date(df, category, deps):
    '''
    DESCRIPTION
        Provide values of the selected category at the last date available in
         the dataframe, for every department.
    INPUT
        category is the name of the columns for which we want values at the 
         last date for every department
    OUPUT
        values is a dictionary with {dep: {'date': date, 'value': value of the
         category at the stored date}}
    '''

    values = dict()

    # format a dedicated list of data for the search
    lst = [np.array(df['dep']),
        np.array(df['date']),
        np.array(df['timestamp']),
        np.array(df[category])]

    # Run over every department
    for dep in deps:
        time_ts = 0
        id_max_date = 0
        # run along the rows of the dataframe
        for i, j in enumerate(lst[0]):
            # When I met the selected department
            #  and the timestamp is higher (ensure get finally the most
            #  recent timestamp/date and related values of the category),
            #  we stored date and value of the category in values, relatively
            #  to the department. 
            if (j == dep) and (time_ts < lst[2][i]):
                id_max_date = i
                time_ts = lst[2][i]

        values[dep] = {'date': lst[1][i], 'value': lst[3][i]}
    return values

# test the function
test_tx = get_values_at_last_date(df, 'tx_test', deps)
print('result in dep 01: {}'.format(test_tx['01'], 3))

result in dep 01: {'date': '2022-12-16', 'value': 0.11546685673556666}


In [274]:
# -----------------------------------------------------------------------------
# DPP4

# Convert result for visualization, I need to get values like
#  date, {dep: values, ...}

def format_values_for_vizu(dico_values, nb_decimal):
    '''
    DESCRIPTION
        format values for a vizualization on a map
    INPUT
        dico_values is a dictionary such {dep: {'date': date, 'value': value
         of the category at the stored date}}
    OUTPUT
        series is a dictionary such as {date1: {depY: valueY, ...}, date2: {...}}
    '''

    series = dict()

    dates = list()
    for dep, j in dico_values.items():
        date = j['date']
        if date not in dates: # Create a sub-dictionary at every new date
            dates.append(date)
            series[date] = dict()
        series[date][dep] = round(j['value'], nb_decimal)

    return series


In [275]:
# -----------------------------------------------------------------------------
# DPP4

# I need to get all values of a categories along the time for a selected department

def get_timeserie(category, dep, duration_week):
    
    # get dataframe the selecte category with time and departments
    df_ = df[['dep', 'date', 'timestamp', category]]

    # keep only rows with the selected dep
    df_ = df_[df_['dep'] == dep]

    # sort dataframe by descending timestamp (most recent at first)
    df_.sort_values(by=['timestamp'], ascending=False, inplace=True)
    
    # get last timestamp (most recent date)
    timestamp_max = df_['timestamp'].values[0]

    # compute start of the time window
    timestamp_min = timestamp_max - 7*duration_week  # unit here is the day
                                                     # 7 days per week

    # keep only dataframe on the required time stamp period
    df_ = df_[df_['timestamp'] > timestamp_min]

    # get expected values and related dates
    dates = df_['date'].values.tolist()
    values = df_[category].values.tolist()
    # revers is for getting time flow from left to right

    return list(reversed(dates)), list(reversed(values))

# -----------------------------------------------------------------------------
Data Post-processing DPP5

Built a prediction model to give 'pos_7j' as target
It may be tested to use dummy values and evaluate if it brings accuracy in the
 result.

In [276]:
# -----------------------------------------------------------------------------
# DPP5
# 
# Try to get dummies on dep
# Since there is a lot, wait and see.

def dummy_data(df, cat):
    '''
    DESCRIPTION
        add dummy data to df removing the source of dummy
    INPUT
        df is the initial dataframe to work on
        cat is the name of the category we want dummying
    OUTPUT
        df_dummy is the dataframe as copy of df but with dummy columns,
         removing the source of dummy
    '''
    df_dummy = df.copy(deep=True)

    # Create a new dataframe for preparing dummies from a copy of the 
    #  selected category 
    df_cat = pd.DataFrame([])
    df_cat[cat] = np.array(df[cat])
    print('shape df_cat:', df_cat.shape)

    # Convert category values to just numbers 0 or 1 (dummy values) of category
    df_dum = pd.get_dummies(df_cat[cat], dtype=int)
    print('shape df_dum:', df_dum.shape)

    for col in df_dum.columns:
        df_dummy[col] = np.array(df_dum[col])
    print('shape df_dummy:', df_dummy.shape)

    # Drop the original category column from `df_dummy`
    df_dummy.drop(columns=cat, inplace=True)
    print('shape df_dummy drop:', df_dummy.shape)

    # check number of duplicates
    #  before trying removing duplicates
    duplicates = df_dummy.duplicated().sum()
    if duplicates > 0:
        df_dummy.drop_duplicates(keep='first', inplace=True)
    print('shape df_dummy dupl:', df_dummy.shape)

    # clean df_dummy from rows with full nan values
    #  otherwise, at least the row is a nana row.
    df_dummy.dropna(axis='index', how='all', inplace=True)
    print('shape df_dummy dropna:', df_dummy.shape)

    return df_dummy


In [277]:
# -----------------------------------------------------------------------------
# DPP5
# 
# Add dummy values of 'dep'

# df = dummy_data(df, 'dep')
# df.head()

In [278]:
# -----------------------------------------------------------------------------
# DPP5
# Check the result

# display_columns_info(df)

In [279]:
# -----------------------------------------------------------------------------
# DPP5
# 
# Clean the dataframe dedicated to model 

# remove the date (str) and dep (str) not supported for
#  the linear regression model.
if 'dep' in df_lin.columns:
    df_lin.drop(columns=['date', 'dep', 'reg', 'dchosp', 'rad', 'incid_rad'], inplace=True)  # without dummies
else:
    df_lin.drop(columns=['date', 'reg', 'dchosp', 'rad', 'incid_rad'], inplace=True)  # with dummies

# Remove unexpected values from the dataframe dedicated to model
df_lin.dropna(axis='index', how='any', inplace=True)

# Check result of the action
print(df_lin.shape)

(95748, 10)


In [280]:
# -----------------------------------------------------------------------------
# DPP5

# Build a model - define the target
cat_response = 'pos_7j'
cat_variables = np.array(df_lin.columns) # useless, included into the get X/y function

In [281]:
# -----------------------------------------------------------------------------
# DPP5

# Build a model - split the dataset into explanatory and response data sets

def get_X_y(df: object, response:str):
    '''
    DESCRIPTION
        Split df into exploratory X data and response y data
    INPUT
           df      : pandas dataframe
           response : target variable
    OUTPUT
           X : A matrix holding all of the variables you want to consider
                when predicting the response
           y : the corresponding response vector
    '''
    # output
    X, y = None, None

    print(" - Spliting data into X/y ...")
    # Get the Response var
    if response in df.columns:
        # try:
        # Split into explanatory and response variables (1/2)
        #  Get response variable
        y = df[response]
        df = df.drop(columns=[response])  # Remove pred_name from df
        # except:
        #     print("    CAUTION: Unable to get the response data in df")
        #     y = None
    else:
        print(" - CAUTION: Unable to find the response in df")
        y = None

    # Get the Exploratory vars
    # try:
    # Split into explanatory and response variables (2/2)
    #  Get the input variables i.e. at this level just a copy of df
    X = df.copy(deep=True)
    # except:
    #     print("    CAUTION: Unable to get the exploratory variables (X)")
    #     X = None

    if X is None:
        print(' - No X found')
    if y is None:
        print(' - No y found')

    del df, response
    return X, y

# Split into Response y / Exploratory variables X
X, y = get_X_y(df_lin, cat_response)

 - Spliting data into X/y ...


In [282]:
# -----------------------------------------------------------------------------
# DPP5

# Build a model - Get a trained model

def get_model(X: object, y: object, testrate=.3):
    '''
    DESCRIPTION
        Returns a linear prediction model according to train data,
        and return r2 scores on train and test data.
    INPUT
           X          : explanatory variables object
           y          : response variable object
           testrate   : proportion of the dataset to include in
                         the test split,between 0.0 and 1.0;
                         default value = 0.3
    OUTPUT
            model : linear regression model object from sklearn
            score : Merge of mean square error value between Train &
                     Test data set according to the proposed model
            list of X_train and y_train
            list of X_test and y_test
    '''
    model, score = None, 0
    Xtrain, Xtest, ytrain, ytest = None, None, None, None

    # sub-variables
    y_pred, msg, split = None, None, False
    acc_score_train, acc_score_test = None, None
    r2_score_train, r2_score_test = None, None
    mdl_score_train, mdl_score_test = None, None
    
    print(" - Spliting data into X/y ...")
    if (X is not None) and (y is not None):

        # Split into train and test X/y data set
        #  to establish the model and score it
        print("\tTraining ...")
        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,
                                                        test_size=testrate,
                                                        random_state=42)
        if (Xtrain is not None) and (Xtest is not None) and \
           (ytrain is not None) and (ytest is not None):
            split = True

        '''
        # Work on dtype
        recensed_type = {}
        for var in X.columns:
            tip = X[var].dtypes
            if tip not in recensed_type:
                recensed_type[tip] = 1
            else:
                recensed_type[tip] = recensed_type[tip] + 1
        '''

        # Establish model
        print("\tModeling ...")
        if split:

            # Linear model from scikit-learn:
            #  https://scikit-learn.org/stable/modules/linear_model.html#

            # Linear Regression
            model = LinearRegression()

            # Linear model Lasso
            # model = linear_model.Lasso(alpha=0.1)

            # Ridge regression and classification
            # model = linear_model.Ridge(alpha=.5)

            # BayesianRidge()
            # model = linear_model.BayesianRidge()

            # LogisticRegression
            # model = linear_model.LogisticRegression(random_state=0)
            # very long -> not achieve

            # generalized linear model TweedieRegressor
            # model = linear_model.TweedieRegressor(power=1, alpha=0.5, link='log')
            # Does not work!

            # Stochastic Gradient Descent
            # model = linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
            # Does not provide good result

            # model  = linear_model.Perceptron(tol=1e-3, random_state=0)
            # Does not provide good result

            # fit the model
            model.fit(Xtrain, ytrain)

            # Test the model on test data
            y_pred = model.predict(Xtest)
        else:
            y_pred = None
            print("\t - No Model defined")
 
    return model, [Xtrain, ytrain], [Xtest, ytest]


# modeling
model, Xy_train, Xy_test = get_model(X, y)

 - Spliting data into X/y ...
	Training ...
	Modeling ...


# -----------------------------------------------------------------------------
Last step of this work consists in building data set for visualization

Visualization required additional processing to get the appropriate values 
 and provide these values in the appropriate format according to the 
 visualization function.
 
Concerning Question 1, 2 and 3, visualization is based on the display of
 results on a map of France, splited by department. Every department as a
 color according to the related value.

Concerning Question 4, visualization consists in a plot of two curves, 
 values of two parameters along the time.
 
Concerning Question 5, visualization consists in display of score and 
 coefficients of the model.

 In the python script, 
Graphical visualizations are displayed on pages of the default web browser.

In [283]:
# -----------------------------------------------------------------------------
# QUESTION 1 - Data set for visualization 

# format for vizualization
tx_test = get_values_at_last_date(df, 'tx_test', deps)
# Get the selected data
tx_test_vizu = format_values_for_vizu(tx_test, 3)
# Data for visualization
print('tx_test_vizu:\n', tx_test_vizu)


tx_test_vizu:
 {'2022-12-16': {'01': 0.115, '02': 0.115, '03': 0.115, '04': 0.115, '05': 0.115, '06': 0.115, '07': 0.115, '08': 0.115, '09': 0.115, '10': 0.115, '11': 0.115, '12': 0.115, '13': 0.115, '14': 0.115, '15': 0.115, '16': 0.115, '17': 0.115, '18': 0.115, '19': 0.115, '21': 0.115, '22': 0.115, '23': 0.115, '24': 0.115, '25': 0.115, '26': 0.115, '27': 0.115, '28': 0.115, '29': 0.115, '2A': 0.115, '2B': 0.115, '30': 0.115, '31': 0.115, '32': 0.115, '33': 0.115, '34': 0.115, '35': 0.115, '36': 0.115, '37': 0.115, '38': 0.115, '39': 0.115, '40': 0.115, '41': 0.115, '42': 0.115, '43': 0.115, '44': 0.115, '45': 0.115, '46': 0.115, '47': 0.115, '48': 0.115, '49': 0.115, '50': 0.115, '51': 0.115, '52': 0.115, '53': 0.115, '54': 0.115, '55': 0.115, '56': 0.115, '57': 0.115, '58': 0.115, '59': 0.115, '60': 0.115, '61': 0.115, '62': 0.115, '63': 0.115, '64': 0.115, '65': 0.115, '66': 0.115, '67': 0.115, '68': 0.115, '69': 0.115, '70': 0.115, '71': 0.115, '72': 0.115, '73': 0.115, '74': 0

Explain the results - Question 1:

	The figures show that COVID-19 testing behaviour among people is rather 
	 homogeneous but low in mainland France while is 2 or 3 times more on 
	 islands in Guyane, Réunion and Mayotte.

In [284]:
# -----------------------------------------------------------------------------
# QUESTION 2 - Data set for visualization

# Get the selected values
tx_incid_IO = get_values_at_last_date(df, 'tx_incid_IO', deps)
# format for vizualization
tx_IO_vizu = format_values_for_vizu(tx_incid_IO, 3)
# Data for visualization
print("tx_IO_vizu:\n", tx_IO_vizu)


tx_IO_vizu:
 {'2022-12-16': {'01': 0.0, '02': 0.0, '03': 0.0, '04': 0.0, '05': 0.0, '06': 0.0, '07': 0.0, '08': 0.0, '09': 0.0, '10': 0.0, '11': 0.0, '12': 0.0, '13': 0.0, '14': 0.0, '15': 0.0, '16': 0.0, '17': 0.0, '18': 0.0, '19': 0.0, '21': 0.0, '22': 0.0, '23': 0.0, '24': 0.0, '25': 0.0, '26': 0.0, '27': 0.0, '28': 0.0, '29': 0.0, '2A': 0.0, '2B': 0.0, '30': 0.0, '31': 0.0, '32': 0.0, '33': 0.0, '34': 0.0, '35': 0.0, '36': 0.0, '37': 0.0, '38': 0.0, '39': 0.0, '40': 0.0, '41': 0.0, '42': 0.0, '43': 0.0, '44': 0.0, '45': 0.0, '46': 0.0, '47': 0.0, '48': 0.0, '49': 0.0, '50': 0.0, '51': 0.0, '52': 0.0, '53': 0.0, '54': 0.0, '55': 0.0, '56': 0.0, '57': 0.0, '58': 0.0, '59': 0.0, '60': 0.0, '61': 0.0, '62': 0.0, '63': 0.0, '64': 0.0, '65': 0.0, '66': 0.0, '67': 0.0, '68': 0.0, '69': 0.0, '70': 0.0, '71': 0.0, '72': 0.0, '73': 0.0, '74': 0.0, '75': 0.0, '76': 0.0, '77': 0.0, '78': 0.0, '79': 0.0, '80': 0.0, '81': 0.0, '82': 0.0, '83': 0.0, '84': 0.0, '85': 0.0, '86': 0.0, '87': 0.0, '88

Explain the results - Question 2:

	Although some departments see the number of patients at hospitals 
	decreasing, the majority of departments slightly increased their number 
	of patients these last 24h.

In [285]:
# -----------------------------------------------------------------------------
# QUESTION 3.1 - Data set for visualization

# Get the selected values
tx_rea = get_values_at_last_date(df, 'tx_rea', deps)
# format for vizualization
tx_rea_vizu = format_values_for_vizu(tx_rea, 3)
# Data for visualization
print("tx_rea_vizu:\n", tx_rea_vizu)

tx_rea_vizu:
 {'2022-12-16': {'01': 0.0, '02': 0.0, '03': 0.0, '04': 0.0, '05': 0.0, '06': 0.0, '07': 0.0, '08': 0.0, '09': 0.0, '10': 0.0, '11': 0.0, '12': 0.0, '13': 0.0, '14': 0.0, '15': 0.0, '16': 0.0, '17': 0.0, '18': 0.0, '19': 0.0, '21': 0.0, '22': 0.0, '23': 0.0, '24': 0.0, '25': 0.0, '26': 0.0, '27': 0.0, '28': 0.0, '29': 0.0, '2A': 0.0, '2B': 0.0, '30': 0.0, '31': 0.0, '32': 0.0, '33': 0.0, '34': 0.0, '35': 0.0, '36': 0.0, '37': 0.0, '38': 0.0, '39': 0.0, '40': 0.0, '41': 0.0, '42': 0.0, '43': 0.0, '44': 0.0, '45': 0.0, '46': 0.0, '47': 0.0, '48': 0.0, '49': 0.0, '50': 0.0, '51': 0.0, '52': 0.0, '53': 0.0, '54': 0.0, '55': 0.0, '56': 0.0, '57': 0.0, '58': 0.0, '59': 0.0, '60': 0.0, '61': 0.0, '62': 0.0, '63': 0.0, '64': 0.0, '65': 0.0, '66': 0.0, '67': 0.0, '68': 0.0, '69': 0.0, '70': 0.0, '71': 0.0, '72': 0.0, '73': 0.0, '74': 0.0, '75': 0.0, '76': 0.0, '77': 0.0, '78': 0.0, '79': 0.0, '80': 0.0, '81': 0.0, '82': 0.0, '83': 0.0, '84': 0.0, '85': 0.0, '86': 0.0, '87': 0.0, '8

In [286]:
# -----------------------------------------------------------------------------
# QUESTION 3.2 - Data set for visualization

# Get the selected values
tx_dchosp = get_values_at_last_date(df, 'tx_dchosp', deps)
# format for vizualization
tx_dchosp_vizu = format_values_for_vizu(tx_dchosp, 3)
# Data for visualization
print("tx_rea_vizu:\n", tx_dchosp_vizu)

tx_rea_vizu:
 {'2022-12-16': {'01': 0.0, '02': 0.0, '03': 0.0, '04': 0.0, '05': 0.0, '06': 0.0, '07': 0.0, '08': 0.0, '09': 0.0, '10': 0.0, '11': 0.0, '12': 0.0, '13': 0.0, '14': 0.0, '15': 0.0, '16': 0.0, '17': 0.0, '18': 0.0, '19': 0.0, '21': 0.0, '22': 0.0, '23': 0.0, '24': 0.0, '25': 0.0, '26': 0.0, '27': 0.0, '28': 0.0, '29': 0.0, '2A': 0.0, '2B': 0.0, '30': 0.0, '31': 0.0, '32': 0.0, '33': 0.0, '34': 0.0, '35': 0.0, '36': 0.0, '37': 0.0, '38': 0.0, '39': 0.0, '40': 0.0, '41': 0.0, '42': 0.0, '43': 0.0, '44': 0.0, '45': 0.0, '46': 0.0, '47': 0.0, '48': 0.0, '49': 0.0, '50': 0.0, '51': 0.0, '52': 0.0, '53': 0.0, '54': 0.0, '55': 0.0, '56': 0.0, '57': 0.0, '58': 0.0, '59': 0.0, '60': 0.0, '61': 0.0, '62': 0.0, '63': 0.0, '64': 0.0, '65': 0.0, '66': 0.0, '67': 0.0, '68': 0.0, '69': 0.0, '70': 0.0, '71': 0.0, '72': 0.0, '73': 0.0, '74': 0.0, '75': 0.0, '76': 0.0, '77': 0.0, '78': 0.0, '79': 0.0, '80': 0.0, '81': 0.0, '82': 0.0, '83': 0.0, '84': 0.0, '85': 0.0, '86': 0.0, '87': 0.0, '8

Explain the results - Question 3:

	The figures show there is some departments with significant level mainly in
     region of Paris, the middle of France, in the South-west and the 
	 North-East, plus The Reunion.

	The figures show a France with a level of decease due to COVID-19 globaly
     low except in some departments of the South-west, North-East and in
	 Guyane with rates between 1.5 and 3 times higher than elsewhere.

In [287]:
# -----------------------------------------------------------------------------
# QUESTION 4 - Data set for visualization

dates_hosp, values_hosp = get_timeserie('hosp', '31', 6*4)
dates_intensive, values_intensive = get_timeserie('rea', '31', 6*4)

# Data for visualization
print("dates_hosp:\n", dates_hosp)
print("values_hosp:\n", values_hosp)
print("values_intensive:\n", values_intensive)

dates_hosp:
 ['2022-07-02', '2022-07-03', '2022-07-04', '2022-07-05', '2022-07-06', '2022-07-07', '2022-07-08', '2022-07-09', '2022-07-10', '2022-07-11', '2022-07-12', '2022-07-13', '2022-07-14', '2022-07-15', '2022-07-16', '2022-07-17', '2022-07-18', '2022-07-19', '2022-07-20', '2022-07-21', '2022-07-22', '2022-07-23', '2022-07-24', '2022-07-25', '2022-07-26', '2022-07-27', '2022-07-28', '2022-07-29', '2022-07-30', '2022-07-31', '2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04', '2022-08-05', '2022-08-06', '2022-08-07', '2022-08-08', '2022-08-09', '2022-08-10', '2022-08-11', '2022-08-12', '2022-08-13', '2022-08-14', '2022-08-15', '2022-08-16', '2022-08-17', '2022-08-18', '2022-08-19', '2022-08-20', '2022-08-21', '2022-08-22', '2022-08-23', '2022-08-24', '2022-08-25', '2022-08-26', '2022-08-27', '2022-08-28', '2022-08-29', '2022-08-30', '2022-08-31', '2022-09-01', '2022-09-02', '2022-09-03', '2022-09-04', '2022-09-05', '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09', '2022-

Explain the results - Question 4:

	In my department of Haute-Garonne, the figures show a continuous increase
     of hospitalization since beginning of November: the level was multiplied
 	 by 1.5 in a month.
	In parallel, the number of deceases at the hospital remains increased in 
	 the same period with a level multiplied by 3 in a month, reaching at last
     40 deceases per day due to COVID-19. This number remains in the result of 
	 these last months.

In [291]:
# -----------------------------------------------------------------------------
# QUESTION 5 - Display performance of the model

def get_model_performance(model, Xy_train, Xy_test, Xy):

    Xtrain, ytrain = Xy_train
    Xtest, ytest = Xy_test
    X, y = Xy


    # Evaluate this model by gettings metrics with a model by Regression
    print("Metrics:")
    
    # Get the model's score
    # Return the coefficient of determination of the prediction
    mdl_score_train = model.score(Xtrain, ytrain)
    print("- Model score (train):", mdl_score_train)
    mdl_score_test = model.score(Xtest, ytest)
    print("- Model score (test):", mdl_score_test)

    # Accuracy_score (https://scikit-learn.org/stable/modules/
    # generated/sklearn.metrics.accuracy_score.html)
    # Confusion matrix (https://scikit-learn.org/stable/modules/
    # generated/sklearn.metrics.confusion_matrix.html)
    #  not appropriate for my purpose
    # Common pitfalls (https://scikit-learn.org/stable/common_
    #  pitfalls.html): mean_sqaured_error and r2_score

    # According to https://stackoverflow.com/questions/37367405/
    #  python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
    # ... Accuracy score is only for classification problems
    # acc_score_train = accuracy_score(y_train, model.predict(X_train))
    # acc_score_test = accuracy_score(y_test, model.predict(X_test))
    # if debug: print("    - acc_score_train:", acc_score_train)
    # if debug: print("    - acc_score_test:", acc_score_test)

    # always according to https://stackoverflow.com/questions/37367405/
    #  python-scikit-learn-cant-handle-mix-of-multiclass-and-continuous
    # For regression problems, use: R2 Score, MSE (Mean Squared Error),
    # RMSE (Root Mean Squared Error).
    r2_score_train = r2_score(ytrain, model.predict(Xtrain))
    r2_score_test = r2_score(ytest, model.predict(Xtest))
    print("- r2 score (train):", r2_score_train)
    print("- r2 score (test):", r2_score_test)

    score = r2_score_train
    print("model's score: ", score)
    
    return score

# Check performance
score = get_model_performance(model, Xy_train, Xy_test, [X, y])

Metrics:
- Model score (train): 0.881512098436752
- Model score (test): 0.8738613401903677
- r2 score (train): 0.881512098436752
- r2 score (test): 0.8738613401903677
model's score:  0.881512098436752


In [289]:
# -----------------------------------------------------------------------------
# QUESTION 5 - Display level of importance of every category on the model

def coef_weights(model, X_train) -> object:
    '''
    returns a dataframe with coefficients of the model
     (real and absolute values) sorted in the descending order
     of the absolute values

    input:
           model     : model for which we are looking coefficients
           X_train   : the training data
    output:
            coefs_df : dataframe with model's coefficients; that can be
                        used to understand the most influential coefficients
                        in a linear model by providing the coefficient
                        estimates along with the name of the variable
                        attached to the coefficient.
    '''
    # function: get model's coefficients

    coefs_df = pd.DataFrame()
    # Get name of every column in front  of its coefficients
    coefs_df['est_int'] = X_train.columns
    # get coefficients of the linear model
    coefs_df['coefs'] = model.coef_
    # get absolute value of these coefficients
    coefs_df['abs_coefs'] = np.abs(model.coef_)
    # Sort coefficient by descending order
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)

    del model, X_train
    return coefs_df


print('  > Define impact of categories on the model:')
coef_df = None

# Compute coefficient of weight on every category for this model
coef_df = coef_weights(model, Xy_train[0])  # Xy_train = [Xtrain, ytrain]
pd.set_option("display.max_rows", None, "display.max_columns", None)
print('\t', coef_df)



  > Define impact of categories on the model:
	         est_int       coefs   abs_coefs
2            TO -966.678578  966.678578
7  incid_dchosp -167.490157  167.490157
6     incid_rea  -75.851722   75.851722
0        tx_pos  -28.293308   28.293308
5    incid_hosp   10.533020   10.533020
4           rea    9.841827    9.841827
3          hosp    4.663104    4.663104
8           pos    4.059020    4.059020
1      tx_incid    3.190276    3.190276


Explain the results - Question 5:

   The model to predict COVID-19 positive cases per week gives rather good
   result
    with a global score of 0.88.
   Analysis of the contribution of every parameters shows that the result is
    mainly linked with the Occupancy rate (TO) which is actuall rather 
	consequence of the prediction.
   It shows a high level a correlation between the number of positive cases
    and the amount of work in the hospitals despite the vaccination campaign.

FINDINGS

About 6% of data were removed from the initial data set due to missing 
 values. It concerns mainly first set of data at the beginning of the data 
 collection. Indeed, method, measure, collection and presentation of data 
 change in time before getting the current format of data.
 Nevertheless, we benefit from a huge amount of valid data for the analysis.
 
Computation of some rates raised the problem of division by 0. While taking 
 about group of people and with the will of showing result for every 
 department, I replace 0 by 1 when necessary.
 
I also face infinite values; then I replaced with infinite values by maximum
 numerical values of the serie, in order to be reasonable realistic and show
 result.

Although testing several type of models, I kept the Linear Regression which
 give rather good results with an implementation rather simple. 
I wonder if some other type of models could be more suitable in this case.

Display of maps and plots from the Notebook was not easy due to difficulties
of installation of additional libraries and other things required to make it 
run correctly; all these installations were not possible on my profressional
laptop to safety policy of my company.
Nevertheless, display of these graph is possible with the python code.